In [1]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\angie\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import TweetTokenizer
from nltk.stem.snowball import SnowballStemmer

In [3]:
df = pd.read_csv("../../data/judge-1377884607_tweet_product_company.csv", encoding = 'latin1')

In [4]:
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [5]:
df = df.dropna(subset=['tweet_text'])

In [6]:
tweets = TweetTokenizer()
df['tokenized'] = df['tweet_text'].apply(tweets.tokenize)
df['tokenized'] = df['tokenized'].str.join(' ')

In [7]:
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,tokenized
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,. @wesley83 I have a 3G iPhone . After 3 hrs t...
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,@jessedee Know about @fludapp ? Awesome iPad /...
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,@swonderlin Can not wait for #iPad 2 also . Th...
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,@sxsw I hope this year's festival isn't as cra...
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,@sxtxstate great stuff on Fri #SXSW : Marissa ...


In [8]:
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,tokenized
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,. @wesley83 I have a 3G iPhone . After 3 hrs t...
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,@jessedee Know about @fludapp ? Awesome iPad /...
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,@swonderlin Can not wait for #iPad 2 also . Th...
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,@sxsw I hope this year's festival isn't as cra...
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,@sxtxstate great stuff on Fri #SXSW : Marissa ...


In [9]:
df.rename(columns = {'emotion_in_tweet_is_directed_at': 'Product', 'is_there_an_emotion_directed_at_a_brand_or_product': 'Emotion', 'tweet_text': 'text'}, inplace= True)

In [10]:
df.head()

,text,Product,Emotion,tokenized
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,. @wesley83 I have a 3G iPhone . After 3 hrs t...
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,@jessedee Know about @fludapp ? Awesome iPad /...
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,@swonderlin Can not wait for #iPad 2 also . Th...
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,@sxsw I hope this year's festival isn't as cra...
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,@sxtxstate great stuff on Fri #SXSW : Marissa ...


In [11]:
df.dropna(subset=['text'], inplace = True)

In [12]:
df['text'] = df['text'].str.replace(r'[^\w\s]', '')
df['text'] = df['text'].str.lower()
df['text'] = df['text'].str.split(' ')
df.fillna('na',inplace=True)
df['test'] = ''

<ipython-input-12-e754b91c759f>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace(r'[^\w\s]', '')


In [13]:
apple = ['iphone','ipad','apple','apples','ipads']
android = ['android','google','androids','googles']
for ind , val in enumerate(df['text']):
    if df['Product'].values[ind] == 'na':
        for y in val:
            if y in apple:
                df['test'].values[ind] = y
            elif y in android:
                df['test'].values[ind] = y
                
    else:
        df['test'].values[ind] = df['Product'].values[ind]

In [14]:
df['test'] = df['test'].map(lambda x : x.lower())
df['test'].unique()

array(['iphone', 'ipad or iphone app', 'ipad', 'google', 'android',
       'apple', 'android app', 'other google product or service', '',
       'googles', 'ipads', 'apples', 'other apple product or service',
       'androids'], dtype=object)

In [15]:
df['test'] = df['test'].replace({'googles': 'google', 'apples': 'apple', 'androids': 'android','ipads':'ipad'})

In [16]:
def company_search(test):
    if test == 'iphone':
        return 'apple'
    elif test == 'apple':
        return 'apple'
    elif test == 'ipad':
        return 'apple'
    elif test == 'ipad or iphone app':
        return 'apple'
    elif test == 'itunes':
        return 'apple'
    elif test == 'other apple product or service':
        return 'apple'
    elif test == 'ios':
        return 'apple'
    elif test == 'ipadquot':
        return 'apple'
    elif test == 'applequot':
        return 'apple'
    elif test == 'iphonequot':
        return 'apple'
    elif test == 'quotiphone':
        return 'apple'
    elif test == 'iosquot':
        return 'apple'
    elif test =='quotipad':
        return 'apple'
    elif test == 'quotapple':
        return 'apple'
    elif test =='google':
        return 'google'
    elif test == 'android':
        return 'google'
    elif test == 'other google product or service':
        return 'google'
    elif test == 'android app':
        return 'google'
    elif test == 'quotgoogle':
        return 'google'
    elif test == 'googlequot':
        return 'google'
    elif test == 'androidquot':
        return 'google'
    else:
        return 'unknown'
df['company'] = df['test'].map(company_search)

In [17]:
df.Emotion.value_counts()

No emotion toward brand or product    5388
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: Emotion, dtype: int64

In [18]:
df['Emotion'] = df['Emotion'].replace({'No emotion toward brand or product': 'neutral', 'I can\'t tell': 'neutral'})

In [19]:
df.head()

,text,Product,Emotion,tokenized,test,company
0,"[wesley83, i, have, a, 3g, iphone, after, 3, h...",iPhone,Negative emotion,. @wesley83 I have a 3G iPhone . After 3 hrs t...,iphone,apple
1,"[jessedee, know, about, fludapp, , awesome, ip...",iPad or iPhone App,Positive emotion,@jessedee Know about @fludapp ? Awesome iPad /...,ipad or iphone app,apple
2,"[swonderlin, can, not, wait, for, ipad, 2, als...",iPad,Positive emotion,@swonderlin Can not wait for #iPad 2 also . Th...,ipad,apple
3,"[sxsw, i, hope, this, years, festival, isnt, a...",iPad or iPhone App,Negative emotion,@sxsw I hope this year's festival isn't as cra...,ipad or iphone app,apple
4,"[sxtxstate, great, stuff, on, fri, sxsw, maris...",Google,Positive emotion,@sxtxstate great stuff on Fri #SXSW : Marissa ...,google,google


In [20]:
df.Emotion.value_counts()

neutral             5544
Positive emotion    2978
Negative emotion     570
Name: Emotion, dtype: int64

In [21]:
# l_encode = LabelEncoder()

In [22]:
df.head()

,text,Product,Emotion,tokenized,test,company
0,"[wesley83, i, have, a, 3g, iphone, after, 3, h...",iPhone,Negative emotion,. @wesley83 I have a 3G iPhone . After 3 hrs t...,iphone,apple
1,"[jessedee, know, about, fludapp, , awesome, ip...",iPad or iPhone App,Positive emotion,@jessedee Know about @fludapp ? Awesome iPad /...,ipad or iphone app,apple
2,"[swonderlin, can, not, wait, for, ipad, 2, als...",iPad,Positive emotion,@swonderlin Can not wait for #iPad 2 also . Th...,ipad,apple
3,"[sxsw, i, hope, this, years, festival, isnt, a...",iPad or iPhone App,Negative emotion,@sxsw I hope this year's festival isn't as cra...,ipad or iphone app,apple
4,"[sxtxstate, great, stuff, on, fri, sxsw, maris...",Google,Positive emotion,@sxtxstate great stuff on Fri #SXSW : Marissa ...,google,google


In [23]:
# emotion_dict = {'No emotion toward brand or product' : 'Neutral emotion', "I can't tell": 'Neutral emotion', 'Positive emotion': 'Positive emotion',
#                'Negative emotion': 'Negative emotion'}
# # df['Emotion'] = df['Emotion'].map(emotion_dict)

In [24]:
df.head()

,text,Product,Emotion,tokenized,test,company
0,"[wesley83, i, have, a, 3g, iphone, after, 3, h...",iPhone,Negative emotion,. @wesley83 I have a 3G iPhone . After 3 hrs t...,iphone,apple
1,"[jessedee, know, about, fludapp, , awesome, ip...",iPad or iPhone App,Positive emotion,@jessedee Know about @fludapp ? Awesome iPad /...,ipad or iphone app,apple
2,"[swonderlin, can, not, wait, for, ipad, 2, als...",iPad,Positive emotion,@swonderlin Can not wait for #iPad 2 also . Th...,ipad,apple
3,"[sxsw, i, hope, this, years, festival, isnt, a...",iPad or iPhone App,Negative emotion,@sxsw I hope this year's festival isn't as cra...,ipad or iphone app,apple
4,"[sxtxstate, great, stuff, on, fri, sxsw, maris...",Google,Positive emotion,@sxtxstate great stuff on Fri #SXSW : Marissa ...,google,google


In [25]:
target_dict = {'Negative emotion': 0, 'neutral': 1, 'Positive emotion': 2}
df['target'] = df['Emotion'].map(target_dict)

In [26]:
df.target.isna().sum()

0

In [27]:
df.Emotion.isna().sum()

0

In [28]:
df['product'] = df['test']

In [29]:
df = df.drop(['Product', 'Emotion', 'test'], axis = 1)

In [30]:
df = df[df['product'] != '']

In [31]:
df.head()

,text,tokenized,company,target,product
0,"[wesley83, i, have, a, 3g, iphone, after, 3, h...",. @wesley83 I have a 3G iPhone . After 3 hrs t...,apple,0,iphone
1,"[jessedee, know, about, fludapp, , awesome, ip...",@jessedee Know about @fludapp ? Awesome iPad /...,apple,2,ipad or iphone app
2,"[swonderlin, can, not, wait, for, ipad, 2, als...",@swonderlin Can not wait for #iPad 2 also . Th...,apple,2,ipad
3,"[sxsw, i, hope, this, years, festival, isnt, a...",@sxsw I hope this year's festival isn't as cra...,apple,0,ipad or iphone app
4,"[sxtxstate, great, stuff, on, fri, sxsw, maris...",@sxtxstate great stuff on Fri #SXSW : Marissa ...,google,2,google


In [32]:
df['text'] = df['text'].str.join(' ')

In [33]:
df.head()

,text,tokenized,company,target,product
0,wesley83 i have a 3g iphone after 3 hrs tweeti...,. @wesley83 I have a 3G iPhone . After 3 hrs t...,apple,0,iphone
1,jessedee know about fludapp awesome ipadiphon...,@jessedee Know about @fludapp ? Awesome iPad /...,apple,2,ipad or iphone app
2,swonderlin can not wait for ipad 2 also they s...,@swonderlin Can not wait for #iPad 2 also . Th...,apple,2,ipad
3,sxsw i hope this years festival isnt as crashy...,@sxsw I hope this year's festival isn't as cra...,apple,0,ipad or iphone app
4,sxtxstate great stuff on fri sxsw marissa maye...,@sxtxstate great stuff on Fri #SXSW : Marissa ...,google,2,google


In [34]:
# snbstem = SnowballStemmer('english')

In [35]:
# df['text'] = df['text'].apply(lambda x: [snbstem.stem(x)])

In [36]:
# df['text'] = df['text'].str.join(' ')

In [37]:
df.head()

,text,tokenized,company,target,product
0,wesley83 i have a 3g iphone after 3 hrs tweeti...,. @wesley83 I have a 3G iPhone . After 3 hrs t...,apple,0,iphone
1,jessedee know about fludapp awesome ipadiphon...,@jessedee Know about @fludapp ? Awesome iPad /...,apple,2,ipad or iphone app
2,swonderlin can not wait for ipad 2 also they s...,@swonderlin Can not wait for #iPad 2 also . Th...,apple,2,ipad
3,sxsw i hope this years festival isnt as crashy...,@sxsw I hope this year's festival isn't as cra...,apple,0,ipad or iphone app
4,sxtxstate great stuff on fri sxsw marissa maye...,@sxtxstate great stuff on Fri #SXSW : Marissa ...,google,2,google


In [38]:
from pickle import dump

In [39]:
dump(df, open('df_clean_tweet_tokens.pkl', 'wb'))

In [40]:
df.target.isna().sum()

0